# Groth16

## Weil pairing

In [1]:
class FiniteField:
    def __init__(self, value, prime):
        if prime <= 1:
            raise ValueError("Prime must be greater than 1")
        self.value = value % prime
        self.prime = prime
        
    def to_extension(self, r):
        """F_p 원소를 F_p^2로 변환"""
        return FiniteFieldExtension(self.value, 0, self.prime, r)
        
    def __add__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value + other.value, self.prime)
    
    def __sub__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value - other.value, self.prime)

    def __mul__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        return FiniteField(self.value * other.value, self.prime)
    
    def __neg__(self):
        return FiniteField(-self.value, self.prime)

    def __truediv__(self, other):
        if not isinstance(other, FiniteField):
            raise TypeError("Operand must be of type FiniteField")
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        if other.value == 0:
            raise ZeroDivisionError("Cannot divide by zero in Finite Field")
        
        return FiniteField(self.value * pow(other.value, -1, self.prime), self.prime)
    
    def __eq__(self, other):
        return isinstance(other, FiniteField) and self.value == other.value and self.prime == other.prime
    
    def __hash__(self):
        """ Make FiniteField hashable so it can be used as a dictionary key """
        return hash((self.value, self.prime))

    def __repr__(self):
        return f"FiniteField({self.value}, {self.prime})"



In [99]:
class FiniteFieldExtension:
    def __init__(self, b, c, prime, r):
        """F_p^2 = F_p[alpha], alpha^2 = r"""
        self.b = FiniteField(b, prime)
        self.c = FiniteField(c, prime)
        self.prime = prime
        self.r = FiniteField(r, prime)
        
        squares = set((i * i) % prime for i in range(prime))
        if self.r.value in squares:
            raise ValueError(f"{r} is a square in F_{prime}, cannot use for extension")
        
    def __add__(self, other:"FiniteFieldExtension") -> "FiniteFieldExtension":
        if self.prime != other.prime:
            raise ValueError("Primes must be the same")
        
        return FiniteFieldExtension(
            (self.b + other.b).value,
            (self.c + other.c).value,
            self.prime,
            self.r.value
        )
        
    def __sub__(self, other):
        if self.prime != other.prime or self.r != other.r:
            raise ValueError("Fields must be the same")
        return FiniteFieldExtension(
            (self.b - other.b).value,
            (self.c - other.c).value,
            self.prime,
            self.r.value
        )
        
    def __mul__(self, other):
        if self.prime != other.prime or self.r != other.r:
            raise ValueError("Fields must be the same")
        b_new = (self.b * other.b + self.c * other.c * self.r).value
        c_new = (self.b * other.c + self.c * other.b).value
        return FiniteFieldExtension(b_new, c_new, self.prime, self.r.value)
    
    def __truediv__(self, other):
        if self.prime != other.prime or self.r != other.r:
            raise ValueError("Fields must be the same")
        if other.b.value == 0 and other.c.value == 0:
            raise ZeroDivisionError("Cannot divide by zero in FiniteFieldExtension")
        
        # Denominator: (other.b + other.c*alpha) * (other.b - other.c*alpha) = other.b^2 - other.c^2 * r
        # denom = other.b * other.b - other.c * other.c * self.r  # Keep as FiniteField
        denom = FiniteFieldExtension((other.b * other.b - other.c * other.c * self.r).value, 0, self.prime, self.r.value)
        
        
        # Numerator: (self.b + self.c*alpha) * (other.b - other.c*alpha) = (self.b*other.b - self.c*other.c*r) + (self.c*other.b - self.b*other.c)*alpha
        num_b = (self.b * other.b - self.c * other.c * self.r).value
        num_c = (self.c * other.b - self.b * other.c).value
        num = FiniteFieldExtension(num_b, num_c, self.prime, self.r.value)
        
        return num * denom.inverse()

    def __neg__(self):
        return FiniteFieldExtension((-self.b).value, (-self.c).value, self.prime, self.r.value)

    def __eq__(self, other):
        return (isinstance(other, FiniteFieldExtension) and
                self.b == other.b and self.c == other.c and
                self.prime == other.prime and self.r == other.r)

    def __hash__(self):
        return hash((self.b.value, self.c.value, self.prime, self.r.value))

    def __repr__(self):
        return f"FiniteFieldExtension({self.b.value} + {self.c.value}*alpha, prime={self.prime}, alpha^2={self.r.value})"
    
    def inverse(self):
        """F_p^2에서 역원을 계산하는 메서드"""
        if self.b.value == 0 and self.c.value == 0:
            raise ZeroDivisionError("Cannot invert zero in FiniteFieldExtension")

        # 분모 계산: (b^2 - c^2 * r) mod p
        denom = FiniteField((self.b * self.b - self.c * self.c * self.r).value, self.prime)

        # 분모의 역원 계산 (pow()를 사용하여 모듈로 역원 구하기)
        denom_inv = FiniteField(pow(denom.value, -1, self.prime), self.prime)

        # 분자의 계수: 켤레(conjugate)를 사용
        b_new = self.b * denom_inv
        c_new = (-self.c) * denom_inv  # c에 -1을 곱함

        return FiniteFieldExtension(b_new.value, c_new.value, self.prime, self.r.value)
        

In [102]:
# F_5 확장체 정의 (α^2 = 2)
a = FiniteFieldExtension(1, 1, 5, 2)
b = FiniteFieldExtension(2, 3, 5, 2)

# 곱셈 테스트
print(f"a * b = {a * b}")

# 역원 테스트
print(f"a^-1 = {a.inverse()}")
print(f"a*a^-1 = {a * a.inverse()}")
# 나눗셈 테스트
print(f"a / b = {a / b}")


a * b = FiniteFieldExtension(3 + 0*alpha, prime=5, alpha^2=2)
a^-1 = FiniteFieldExtension(4 + 1*alpha, prime=5, alpha^2=2)
a*a^-1 = FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)
a / b = FiniteFieldExtension(1 + 4*alpha, prime=5, alpha^2=2)


In [113]:
class EllipticCurve:
    def __init__(self, a, b, prime):
        '''y^2 = x^3 + ax + b mod prime'''
        self.a = a
        self.b = b
        self.prime = prime
        self.infinity = None
        self.points = [self.infinity]

        if isinstance(a, FiniteField) and isinstance(b, FiniteField):
            self.field_type = "FiniteField"
            self.r = None
            self.discriminant = (FiniteField(4, self.prime) * (self.a * self.a * self.a) + 
                                FiniteField(27, self.prime) * (self.b * self.b)).value
            if self.discriminant % self.prime == 0:
                raise ValueError("This curve is not elliptic (discriminant is zero)")

            for x_val in range(self.prime):
                for y_val in range(self.prime):
                    x = FiniteField(x_val, self.prime)
                    y = FiniteField(y_val, self.prime)
                    if self.is_point_on_curve(x, y):
                        self.points.append((x, y))
                        
        elif isinstance(a, FiniteFieldExtension) and isinstance(b, FiniteFieldExtension):
            self.field_type = "FiniteFieldExtension"
            self.r = a.r.value
            self.discriminant = (FiniteFieldExtension(4, 0, self.prime, self.r) * (self.a * self.a * self.a) + 
                                FiniteFieldExtension(27, 0, self.prime, self.r) * (self.b * self.b))
            if self.discriminant.b.value == 0 and self.discriminant.c.value == 0:
                raise ValueError("This curve is not elliptic (discriminant is zero)")

            squares = set()
            for b_val_y in range(self.prime):
                for c_val_y in range(self.prime):
                    y = FiniteFieldExtension(b_val_y, c_val_y, self.prime, self.r)
                    y2 = y * y
                    squares.add((y2.b.value, y2.c.value))

            for b_val in range(self.prime):
                for c_val in range(self.prime):
                    x = FiniteFieldExtension(b_val, c_val, self.prime, self.r)
                    rhs = x * x * x + self.a * x + self.b
                    if (rhs.b.value, rhs.c.value) in squares:
                        for b_val_y in range(self.prime):
                            for c_val_y in range(self.prime):
                                y = FiniteFieldExtension(b_val_y, c_val_y, self.prime, self.r)
                                lhs = y * y
                                if lhs.b.value == rhs.b.value and lhs.c.value == rhs.c.value:
                                    self.points.append((x, y))
        else:
            raise ValueError("a and b must both be either FiniteField or FiniteFieldExtension instances")

    def is_point_on_curve(self, x, y):
        left = (y * y)
        right = (x * x * x + self.a * x + self.b)
        if self.field_type == "FiniteField":
            return left.value == right.value % self.prime
        else:
            return left.b.value == right.b.value and left.c.value == right.c.value

    def add_points(self, p1, p2):
        if p1 not in self.points or p2 not in self.points:
            raise ValueError(f"Points are not on the curve: p1={p1}, p2={p2}")
        
        if p1 is self.infinity:
            return p2
        if p2 is self.infinity:
            return p1     
        
        x1, y1 = p1
        x2, y2 = p2
        
        if x1 == x2 and y1 != y2:
            return self.infinity
        
        if p1 == p2:
            if (isinstance(y1, FiniteField) and y1.value == 0) or \
            (isinstance(y1, FiniteFieldExtension) and y1.b.value == 0 and y1.c.value == 0):
                return self.infinity
            if self.field_type == "FiniteField":
                slope = (FiniteField(3, self.prime) * x1 * x1 + self.a) / (FiniteField(2, self.prime) * y1)
            else:
                three = FiniteFieldExtension(3, 0, self.prime, self.r)
                two = FiniteFieldExtension(2, 0, self.prime, self.r)
                slope = (three * x1 * x1 + self.a) / (two * y1)
        else:
            if x1 == x2:
                return self.infinity
            slope = (y2 - y1) / (x2 - x1)
            
        x3 = slope * slope - x1 - x2
        y3 = slope * (x1 - x3) - y1
        if self.field_type == "FiniteFieldExtension":
            x3 = FiniteFieldExtension(x3.b.value % self.prime, x3.c.value % self.prime, self.prime, self.r)
            y3 = FiniteFieldExtension(y3.b.value % self.prime, y3.c.value % self.prime, self.prime, self.r)
            
        result = (x3, y3)
        if not self.is_point_on_curve(x3, y3):
            raise ValueError(f"Invalid add. Add result is not in the curve: {result}.\n p1:{p1}, p2:{p2} \n points: {self.points}")
        return result
    
    def multiply_point(self, p, n):
        if p is None:
            return self.infinity  # None 대신 항등원 반환
        if n == 0:
            return self.infinity  # 0배는 항등원
        if n < 0:
            return self.multiply_point((-p[0], -p[1]), -n)  # 음수 스칼라 지원
        
        result = self.infinity  # 초기값을 항등원으로 설정
        temp = p
        n = n % self.prime  # 모듈로 p로 줄임 (선택 사항, 큰 n 방지)
        
        while n > 0:
            if n % 2 == 1:
                result = self.add_points(result, temp)
            temp = self.add_points(temp, temp)
            n //= 2
        return result
    
    def find_points_of_order_n(self, n):
        result = []
        for point in self.points:
            if point is None:
                continue
            if self.multiply_point(point, n) == self.infinity:
                order = 1
                current = point
                while current != self.infinity and order <= n:
                    current = self.add_points(current, point)
                    order += 1
                if order == n:
                    result.append(point)
        return result
    
    def is_linear_independent(self, p1, p2, n=6):
        if p1 == self.infinity or p2 == self.infinity:
            return False
        
        # p1을 F_p^2로 확장
        x1, y1 = p1
        if isinstance(x1, FiniteField) and isinstance(y1, FiniteField):
            extended_p1 = (x1.to_extension(self.r), y1.to_extension(self.r))
        else:
            extended_p1 = p1
        
        # a*p1 + b*p2 = O 확인
        for a in range(n):
            for b in range(n):
                if a == 0 and b == 0:
                    continue
                if self.add_points(self.multiply_point(extended_p1, a), self.multiply_point(p2, b)) == self.infinity:
                    return False
        return True
    
    def __repr__(self):
        return f"EllipticCurve(y^2 = x^3 + {self.a}x + {self.b} over F_{self.prime})"
		

In [114]:
curve_q = EllipticCurve(a=FiniteFieldExtension(0, 0, 5, 2), b=FiniteFieldExtension(1, 0, 5, 2), prime=5)
points = curve_q.points
add_result = curve_q.add_points(points[9], points[9])
print(curve_q.points)
print(points[9])
print(add_result)

[None, (FiniteFieldExtension(0 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(0 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(4 + 0*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(0 + 2*alpha, prime=5, alpha^2=2), FiniteFieldExtension(2 + 4*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(0 + 2*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 1*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(0 + 3*alpha, prime=5, alpha^2=2), FiniteFieldExtension(2 + 1*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(0 + 3*alpha, prime=5, alpha^2=2), FiniteFieldExtension(3 + 4*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(0 + 1*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(1 + 0*alpha, prime=5, alpha^2=2), FiniteFieldExtension(0 + 4*alpha, prime=5, alpha^2=2)), (FiniteFieldExtension(1 + 1*alpha, prime=5, alpha^2=2), FiniteFieldExtension(0 + 2*alpha, prime=5

In [115]:
from collections import Counter

class Divisor:
    def __init__(self, curve: EllipticCurve):
        self.curve = curve
        self.points = curve.points
        self.prime = curve.prime
        self.r = self.points[1][0].r.value
        
    def generate_equiv_divisors(self, point, point_R):
        """(P) - (O)와 동등한 divisor 생성: (P + R) - (R)"""
        P_plus_R = self.curve.add_points(point, point_R)
        divisor = Counter()
        divisor[P_plus_R] = 1
        divisor[point_R] = -1
        return divisor
    
    def compute_divisor_of_function(self, function, is_vertical=False, is_tangent=False):
        divisor = Counter()
        a, b, c = function
        number_of_zeros = 0

        # 영점 계산
        for point in self.points:
            if point == self.curve.infinity:
                continue
            x, y = point
            value = a * x + b * y + c
            if value.b.value == 0 and value.c.value == 0:  # FiniteFieldExtension에서 0 판단
                divisor[point] += 1  # 영점 계수 증가
                number_of_zeros += 1

        # 극점 차수 설정
        if is_vertical:
            # 수직선의 경우 극점 차수는 -2 (타원 곡선과 교점이 2개)
            divisor[self.curve.infinity] = -2
        elif is_tangent:
            # 접선의 경우 극점 차수는 -3 (중복 교점 포함)
            divisor[self.curve.infinity] = -3
        else:
            # 일반 직선의 경우 극점 차수는 -3 (타원 곡선과 교점이 3개)
            divisor[self.curve.infinity] = -3

        return divisor
            
    def evaluate_function(self, divisor, linear_function, is_vertical=False, is_tangent=False):
        # 직선 함수의 divisor 계산
        #function_divisor = self.compute_divisor_of_function(linear_function, is_vertical, is_tangent)
        result = FiniteFieldExtension(1, 0, self.curve.prime, self.r)
        a, b, c = linear_function
        
        # divisor에 따라 값 평가
        for point, exponent in divisor.items():
            if point == self.curve.infinity:
                continue
            x, y = point
            value = a * x + b * y + c
            #print(f"Evaluating at {point}: {value} (function divisor at {point} = {function_divisor[point]})")
            if value.b.value == 0 and value.c.value == 0:
                raise ValueError("Function evaluation resulted in zero, which may cause invalid pairing.")
            if exponent > 0:
                for _ in range(exponent):
                    result = result * value
            elif exponent < 0:
                value_inv = value.inverse()
                for _ in range(-exponent):
                    result = result * value_inv
        
        # 극점 처리 (function_divisor의 극점 차수 반영)
        ''' 
            if function_divisor[self.curve.infinity] < 0:
                value_at_infinity = FiniteFieldExtension(1, 0, self.curve.prime, self.r)  # 극점에서의 값은 1로 가정
                for _ in range(-function_divisor[self.curve.infinity]):
                    result = result / value_at_infinity
            '''
        return result

In [116]:
class WeilPairing:
    def __init__(self, curve: EllipticCurve, P, Q, R_P=None, R_Q=None):
            self.curve = curve
            self.P = P
            self.Q = Q
            self.r = self.curve.points[1][0].r.value
            if P not in curve.points or Q not in curve.points:
                raise ValueError("Points are not on the curve")
            self.div = Divisor(curve)
            self.R_P = R_P
            self.R_Q = R_Q
            
    def line_through_vertical(self, P):
        # 먼저 무한대점인지 확인
        if P == self.curve.infinity:
                return (FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                        FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                        FiniteFieldExtension(1, 0, self.curve.prime, self.r))  # f = 1
        # 그 다음에 언팩
        x, y = P
        return (FiniteFieldExtension(1, 0, self.curve.prime, self.r),
                FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                -x)
    
    def line_through_points(self, P, Q):
        # 먼저 무한대점인지 확인
        if P == self.curve.infinity or Q == self.curve.infinity:
                return (FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                        FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                        FiniteFieldExtension(1, 0, self.curve.prime, self.r))  # f = 1
        
        # 그 다음에 언팩
        x1, y1 = P
        x2, y2 = Q
        
        # 나머지 로직은 동일
        if P == Q:
            if (isinstance(y1, FiniteField) and y1.value == 0) or \
            (isinstance(y1, FiniteFieldExtension) and y1.b.value == 0 and y1.c.value == 0):
                return (FiniteFieldExtension(1, 0, self.curve.prime, self.r),
                        FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                        -x1)
            slope = (FiniteFieldExtension(3, 0, self.curve.prime, self.r) * x1 * x1 + self.curve.a) / \
                    (FiniteFieldExtension(2, 0, self.curve.prime, self.r) * y1)
            a = -slope
            b = FiniteFieldExtension(1, 0, self.curve.prime, self.r)
            c = slope * x1 - y1
            return (a, b, c)
        else:
            if x1 == x2:
                return (FiniteFieldExtension(1, 0, self.curve.prime, self.r),
                        FiniteFieldExtension(0, 0, self.curve.prime, self.r),
                        -x1)
            slope = (y2 - y1) / (x2 - x1)
            a = -slope
            b = FiniteFieldExtension(1, 0, self.curve.prime, self.r)
            c = y1 - slope * x1
            return (a, b, c)
                
    def miller(self, order, point, divisor):
        if point == self.curve.infinity:
            return FiniteFieldExtension(1, 0, self.curve.prime, self.curve.r)  # e(O, Q) = 1
        bin_order = bin(order)[2:]
        Z = point
        V = FiniteFieldExtension(1, 0, self.curve.prime, self.curve.r)
        
        for i in range(len(bin_order)):
            #print(f"Step {i}, Before Double: Z = {Z}, V = {V}")
            if Z == self.curve.infinity:
                continue  # Z가 O일 때는 계산 건너뛰기
            numerator = self.div.evaluate_function(divisor, self.line_through_points(Z, Z))
            denominator = self.div.evaluate_function(divisor, self.line_through_vertical(self.curve.add_points(Z, Z)))
            #print(f"numerator = {numerator}, denominator = {denominator}")
            V = V * V * (numerator / denominator)
            #print(f"After Double: V = {V}")
            Z = self.curve.add_points(Z, Z)
            if bin_order[i] == '1':
                #print(f"Step {i}, Before Add: Z = {Z}, V = {V}")
                if Z == self.curve.infinity:
                    continue  # Z가 O일 때는 계산 건너뛰기
                Z = self.curve.add_points(Z, point)
                if Z == self.curve.infinity:
                    continue
                numerator = self.div.evaluate_function(divisor, self.line_through_points(Z, point))
                denominator = self.div.evaluate_function(divisor, self.line_through_vertical(Z))
                #print(f"numerator = {numerator}, denominator = {denominator}")
                V = V * (numerator / denominator)
                #print(f"After Add: V = {V}")
        return V

    def compute_pairing(self, order):
        if self.P == self.curve.infinity or self.Q == self.curve.infinity:
            return FiniteFieldExtension(1, 0, self.curve.prime, self.curve.r), None, None
        if self.P == self.Q:
            return FiniteFieldExtension(1, 0, self.curve.prime, self.curve.r), None, None
        
        max_attempts = 50
        for _ in range(max_attempts):
            try:
                if self.R_P is None:
                    R_P = random.choice([pt for pt in self.curve.points if pt != self.P and pt != self.curve.infinity])
                else: 
                    R_P = self.R_P
                
                if self.R_Q is None:
                    R_Q = random.choice([pt for pt in self.curve.points if pt != self.Q and pt != self.curve.infinity])
                else:
                    R_Q = self.R_Q

                divisor_P = self.div.generate_equiv_divisors(self.P, R_P)
                divisor_Q = self.div.generate_equiv_divisors(self.Q, R_Q)
                f_P_A_Q = self.miller(order, self.P, divisor_Q)
                f_Q_A_P = self.miller(order, self.Q, divisor_P)
                result = f_P_A_Q / f_Q_A_P
                # 추가 검증: 결과가 0이 아닌지 확인
                if result.b.value != 0 or result.c.value != 0:
                    # order가 n인지 확인
                    power_n = result
                    for i in range(order - 1):
                        power_n = power_n * result
                    if power_n == FiniteFieldExtension(1, 0, self.curve.prime, self.curve.r):
                        return result, R_P, R_Q
            except ValueError:
                continue
        raise ValueError("Failed to compute pairing after multiple attempts")


In [135]:
curve_p = EllipticCurve(a=FiniteField(1, 17), b=FiniteField(0, 17), prime=17)
curve_q = EllipticCurve(a=FiniteFieldExtension(1, 0, 17, 5), b=FiniteFieldExtension(0, 0, 17, 5), prime=17)

points_p_of_order_6 = [p for p in curve_p.find_points_of_order_n(4) if p is not None]
points_q_of_order_6 = [p for p in curve_q.find_points_of_order_n(4) if p is not None]

print(points_p_of_order_6)
print(points_q_of_order_6)

[(FiniteField(1, 17), FiniteField(6, 17)), (FiniteField(1, 17), FiniteField(11, 17)), (FiniteField(3, 17), FiniteField(8, 17)), (FiniteField(3, 17), FiniteField(9, 17)), (FiniteField(6, 17), FiniteField(1, 17)), (FiniteField(6, 17), FiniteField(16, 17)), (FiniteField(11, 17), FiniteField(4, 17)), (FiniteField(11, 17), FiniteField(13, 17)), (FiniteField(14, 17), FiniteField(2, 17)), (FiniteField(14, 17), FiniteField(15, 17)), (FiniteField(16, 17), FiniteField(7, 17)), (FiniteField(16, 17), FiniteField(10, 17))]
[(FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5), FiniteFieldExtension(6 + 0*alpha, prime=17, alpha^2=5)), (FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5), FiniteFieldExtension(11 + 0*alpha, prime=17, alpha^2=5)), (FiniteFieldExtension(3 + 0*alpha, prime=17, alpha^2=5), FiniteFieldExtension(8 + 0*alpha, prime=17, alpha^2=5)), (FiniteFieldExtension(3 + 0*alpha, prime=17, alpha^2=5), FiniteFieldExtension(9 + 0*alpha, prime=17, alpha^2=5)), (FiniteFieldExtension(6 + 

In [160]:
import random
curve_p = EllipticCurve(a=FiniteField(1, 17), b=FiniteField(0, 17), prime=17)
curve_q = EllipticCurve(a=FiniteFieldExtension(1, 0, 17, 5), b=FiniteFieldExtension(0, 0, 17, 5), prime=17)
extension = 5
points_p_of_order_6 = [p for p in curve_p.find_points_of_order_n(4) if p is not None]
points_q_of_order_6 = [p for p in curve_q.find_points_of_order_n(4) if p is not None]

if not points_p_of_order_6:
    raise ValueError("No point of order 6 found in E(F_p). Change order n")
if not points_q_of_order_6:
    raise ValueError("No point of order 6 found in E(F_p^2). Change order n")

point_P = random.choice(points_p_of_order_6)
extended_point_P = (point_P[0].to_extension(extension), point_P[1].to_extension(extension))
point_Q = random.choice(points_q_of_order_6)
    
    
def test_properties_nth_root(curve_q, point_P, point_Q):
    print("Testing properties of Weil Pairing...")
    print(f"Curve: {curve_q}")
    print(f"Point P: {point_P}")
    print(f"Point Q: {point_Q}")
    # 선형 독립성 확인
    '''
    if not curve_q.is_linear_independent(point_P, point_Q):
        print("Points are linearly dependent, choosing new Q...")
        for q in points_q_of_order_6:
            if curve_q.is_linear_independent(point_P, q):
                point_Q = q
                break
        else:
            raise ValueError("Could not find linearly independent Q")
    '''
    weil_pairing = WeilPairing(curve_q, extended_point_P, point_Q)
    result, R_P, R_Q = weil_pairing.compute_pairing(6)
    print(f'{1}th order: {result}')
    power_6 = result
    print("----------------")
    for i in range(1, 6):
        power_6 = power_6 * result
        print(f'{i+1}th order: {power_6}')        
    print("----------------")    
    assert power_6 == FiniteFieldExtension(1, 0, 17, extension), f"Identity property failed: e(P, Q)^6 = {power_6} != 1"
    print("Identity property test passed!")
    
    weil_pairing = WeilPairing(curve_q, extended_point_P, point_Q, R_P, R_Q)
    result, R_P, R_Q = weil_pairing.compute_pairing(6)
    print(f'{1}th order: {result}')
    power_6 = result
    print("----------------")
    for i in range(1, 6):
        power_6 = power_6 * result
        print(f'{i+1}th order: {power_6}')        
    print("----------------")    
    assert power_6 == FiniteFieldExtension(1, 0, 17, extension), f"Identity property failed: e(P, Q)^6 = {power_6} != 1"
    print("Identity property test passed!")

In [161]:
test_properties_nth_root(curve_q, point_P, point_Q)

Testing properties of Weil Pairing...
Curve: EllipticCurve(y^2 = x^3 + FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5)x + FiniteFieldExtension(0 + 0*alpha, prime=17, alpha^2=5) over F_17)
Point P: (FiniteField(6, 17), FiniteField(16, 17))
Point Q: (FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5), FiniteFieldExtension(11 + 0*alpha, prime=17, alpha^2=5))
1th order: FiniteFieldExtension(8 + 13*alpha, prime=17, alpha^2=5)
----------------
2th order: FiniteFieldExtension(8 + 4*alpha, prime=17, alpha^2=5)
3th order: FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5)
4th order: FiniteFieldExtension(8 + 13*alpha, prime=17, alpha^2=5)
5th order: FiniteFieldExtension(8 + 4*alpha, prime=17, alpha^2=5)
6th order: FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5)
----------------
Identity property test passed!
1th order: FiniteFieldExtension(8 + 13*alpha, prime=17, alpha^2=5)
----------------
2th order: FiniteFieldExtension(8 + 4*alpha, prime=17, alpha^2=5)
3th order: FiniteF

In [164]:
def test_properties_equal_points():
    prime = 17
    curve_p = EllipticCurve(a=FiniteField(1, prime), b=FiniteField(0, prime), prime=prime)
    curve_q = EllipticCurve(a=FiniteFieldExtension(1, 0, prime, 5), b=FiniteFieldExtension(0, 0, prime, 5), prime=prime)
    extension = 5
    points_p_of_order_6 = [p for p in curve_p.find_points_of_order_n(4) if p is not None]
    points_q_of_order_6 = [p for p in curve_q.find_points_of_order_n(4) if p is not None]
    
    if not points_p_of_order_6:
        raise ValueError("No point of order 6 found in E(F_p). Change order n")
    
    if not points_q_of_order_6:
        raise ValueError("No point of order 6 found in E(F_p^2). Change order n")
    
    # 다양한 점 테스트
    for point_on_p in points_p_of_order_6:
        extended_point_p = (point_on_p[0].to_extension(extension), point_on_p[1].to_extension(extension))
        weil = WeilPairing(curve_q, extended_point_p, extended_point_p)
        result, _, _ = weil.compute_pairing(6)
        assert result == FiniteFieldExtension(1, 0, prime, extension), f"Failed for point {point_on_p}"
    
    for point_on_q in points_q_of_order_6:
        weil = WeilPairing(curve_q, point_on_q, point_on_q)
        result, _, _ = weil.compute_pairing(6)
        assert result == FiniteFieldExtension(1, 0, prime, extension), f"Failed for point {point_on_q}"

In [165]:
test_properties_equal_points()

In [618]:
# p = 5, n = 3 또는 n=6 일떄 서로 다른 P를 쓸경우 (P1, P2, Q)가 선형 독립인 조합 찾기 어려움
# 따라서 P2 = O로 설정
def test_properties_bilinear():
    prime = 17
    extension = 3
    
    curve_p = EllipticCurve(a=FiniteField(1, prime), b=FiniteField(0, prime), prime=prime)
    curve_q = EllipticCurve(a=FiniteFieldExtension(1, 0, prime, extension), b=FiniteFieldExtension(0, 0, prime, extension), prime=prime)
    n = 4
    points_p_of_order_n = [p for p in curve_p.find_points_of_order_n(n) if p is not None]
    points_q_of_order_n = [p for p in curve_q.find_points_of_order_n(n) if p is not None]
    
    
    if not points_p_of_order_n:
        raise ValueError("No point of order 6 found in E(F_p). Change order n")
    if not points_q_of_order_n:
        raise ValueError("No point of order 6 found in E(F_p^2). Change order n")
    
    point_P = random.choice(points_p_of_order_n)
    extended_point_P = (point_P[0].to_extension(extension), point_P[1].to_extension(extension))
    extended_added_point_P = curve_q.add_points(extended_point_P, curve_q.infinity)
    point_Q = random.choice(points_q_of_order_n)
    '''
    if not (curve_q.is_linear_independent(extended_added_point_P, point_Q) and
            curve_q.is_linear_independent(extended_point_P, point_Q)):
        print("Points are linearly dependent, choosing new Q...")
        for q in points_q_of_order_n:
            if (curve_q.is_linear_independent(extended_added_point_P, q) and
                curve_q.is_linear_independent(extended_point_P, q)):
                point_Q = q
                break
        else:
            raise ValueError("Could not find linearly independent Q. Please run the test again to select new points.")
    '''    
    print(f"P = {extended_point_P}")
    print(f"Q = {point_Q}")
    print(f"P + Q = {extended_added_point_P}") 
    
    weil_pairing_lhs = WeilPairing(curve_q, extended_added_point_P, point_Q)
    lhs, R_eaP, R_Q = weil_pairing_lhs.compute_pairing(n)
    
    weil_pairing_rhs1 = WeilPairing(curve_q, extended_point_P, point_Q, R_eaP, R_Q)
    weil_pairing_rhs2 = WeilPairing(curve_q, curve_q.infinity, point_Q, None, R_Q)
    
    rhs1, _, R_rhsQ = weil_pairing_rhs1.compute_pairing(n)
    rhs2, _, _ = weil_pairing_rhs2.compute_pairing(n)
    rhs = rhs1 * rhs2
    print("----------------")
    print(f"e(P+Q, Q) = {lhs}, e(P, Q) = {rhs1}, e(Q, Q) = {rhs2}")
    print(f"lhs = e(P+O, Q) = {lhs}, rhs = e(P, Q) * e(Q, Q) = {rhs}, lhs == rhs = {lhs == rhs}")
    assert R_Q == R_rhsQ, f"Invalid R_Q lhs R_Q={R_Q}, rhs R_Q={R_rhsQ}"
    assert lhs == rhs, f"Failed for bilinear test lhs={lhs}, rhs={rhs}"

In [619]:
test_properties_bilinear()

P = (FiniteFieldExtension(6 + 0*alpha, prime=17, alpha^2=3), FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=3))
Q = (FiniteFieldExtension(3 + 0*alpha, prime=17, alpha^2=3), FiniteFieldExtension(9 + 0*alpha, prime=17, alpha^2=3))
P + Q = (FiniteFieldExtension(6 + 0*alpha, prime=17, alpha^2=3), FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=3))
----------------
e(P+Q, Q) = FiniteFieldExtension(16 + 0*alpha, prime=17, alpha^2=3), e(P, Q) = FiniteFieldExtension(16 + 0*alpha, prime=17, alpha^2=3), e(Q, Q) = FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=3)
lhs = e(P+O, Q) = FiniteFieldExtension(16 + 0*alpha, prime=17, alpha^2=3), rhs = e(P, Q) * e(Q, Q) = FiniteFieldExtension(16 + 0*alpha, prime=17, alpha^2=3), lhs == rhs = True


In [180]:
import random
prime = 17
curve_p = EllipticCurve(a=FiniteField(1, prime), b=FiniteField(0, prime), prime=prime)
curve_q = EllipticCurve(a=FiniteFieldExtension(1, 0, prime, 5), b=FiniteFieldExtension(0, 0, prime, 5), prime=prime)
extension = 5
n = 4
points_p_of_order_n = [p for p in curve_p.find_points_of_order_n(n) if p is not None]
points_q_of_order_n = [p for p in curve_q.find_points_of_order_n(n) if p is not None]

if not points_p_of_order_n:
    raise ValueError("No point of order n found in E(F_p). Change order n")
if not points_q_of_order_n:
    raise ValueError("No point of order n found in E(F_p^2). Change order n")

point_P = random.choice(points_p_of_order_n)
extended_point_P = (point_P[0].to_extension(extension), point_P[1].to_extension(extension))
point_Q = random.choice(points_q_of_order_n)
    
    
def test_properties_alternative(curve_q, point_P, point_Q):
    print("Testing properties of Weil Pairing...")
    print(f"Curve: {curve_q}")
    print(f"Point P: {point_P}")
    print(f"Point Q: {point_Q}")
    # 선형 독립성 확인
    '''
    if not curve_q.is_linear_independent(point_P, point_Q):
        print("Points are linearly dependent, choosing new Q...")
        for q in points_q_of_order_6:
            if curve_q.is_linear_independent(point_P, q):
                point_Q = q
                break
        else:
            raise ValueError("Could not find linearly independent Q")
    '''
    weil_pairing1 = WeilPairing(curve_q, extended_point_P, point_Q)
    result1, R_P, R_Q = weil_pairing1.compute_pairing(6)
    
    weil_pairing2 = WeilPairing(curve_q, point_Q, extended_point_P, R_Q, R_P)
    result2, _, __Q = weil_pairing2.compute_pairing(6)
    
    print(f'{result1}')
    print(f'{result2}')
    print(f'Alternative {result1*result2} should be 1 beacsue result1 and result2 are inverse')


In [181]:
test_properties_alternative(curve_q, point_P, point_Q)

Testing properties of Weil Pairing...
Curve: EllipticCurve(y^2 = x^3 + FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5)x + FiniteFieldExtension(0 + 0*alpha, prime=17, alpha^2=5) over F_17)
Point P: (FiniteField(14, 17), FiniteField(15, 17))
Point Q: (FiniteFieldExtension(14 + 0*alpha, prime=17, alpha^2=5), FiniteFieldExtension(15 + 0*alpha, prime=17, alpha^2=5))
FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5)
FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5)
Alternative FiniteFieldExtension(1 + 0*alpha, prime=17, alpha^2=5) should be 1 beacsue result1 and result2 are inverse


## QAP

In [182]:
import numpy as np
import sympy as sp

In [183]:
class Node:
    def __init__(self, operation=None, left=None, right=None, value=None):
        self.operation = operation
        self.left = left
        self.right = right
        self.value = value
        
    def evaluate(self):
        if self.operation is None:
            return self.value
        elif self.operation == '+':
            return self.left.evaluate() + self.right.evaluate()
        elif self.operation == '*':
            return self.left.evaluate() * self.right.evaluate()
        
        
    def __repr__(self):
        """노드 정보 보기 쉽게 출력"""
        if self.operation is None:
            return f"({self.value})"
        return f"({self.left} {self.operation} {self.right})"
        

In [184]:
def find_left_selector(gate, gate_idx, c, selector):    
    if gate.operation == None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")    
    elif gate.left.operation == None:
        if gate.left.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")
    elif gate.left.operation == '+':
        find_left_selector(gate.left.left, gate_idx, c, selector)
        find_left_selector(gate.left.right, gate_idx, c, selector)
    elif gate.operation == '*':
        if gate.left.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")

    return selector

def find_right_selector(gate, gate_idx, c, selector):    
    if gate.operation == None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")    
    elif gate.right.operation == None:
        if gate.right.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")
    elif gate.right.operation == '+':
        find_right_selector(gate.right.right, gate_idx, c, selector)
        find_right_selector(gate.right.left, gate_idx, c, selector)
    elif gate.operation == '*':
        if gate.right.evaluate() == c:
            selector[gate_idx] = 1        
            print(f"gate_idx: {gate_idx}, c: {c}, selector: {selector}")

    return selector

    '''
def gate_selector(gate_list, direction, transcript_table):
    polynomial_table = []
    if direction == 'left':
        for c in transcript_table:
            selector = [0] * len(gate_list)
            for gate_idx, gate in enumerate(gate_list):
                find_left_selector(gate, gate_idx, c, selector)
            polynomial_table.append(selector)
            
    elif direction == 'right':
        for c in transcript_table:
            selector = [0] * len(gate_list)
            for gate_idx, gate in enumerate(gate_list):
                find_right_selector(gate, gate_idx, c, selector)
            polynomial_table.append(selector)
            
    return polynomial_table
    '''

In [185]:
def find_selector(gate, gate_idx, c, selector, direction):
    """Left 또는 Right Selector Polynomial을 찾는 함수"""
    if gate is None:
        return
    
    # 리프 노드라면 값을 확인
    if gate.operation is None:
        if gate.evaluate() == c:
            selector[gate_idx] = 1        
        return
    
    # 왼쪽 / 오른쪽 / 출력 선택
    if direction == 'left':
        target = gate.left 
    elif direction == 'right':
        target = gate.right
    elif direction == 'output':
        target = gate

    # 리프 값이 맞는 경우
    if target and target.operation is None and target.evaluate() == c:
        selector[gate_idx] = 1
        return

    # 덧셈 노드인 경우 재귀 호출
    if target and target.operation == '+':
        find_selector(target.left, gate_idx, c, selector, direction)
        find_selector(target.right, gate_idx, c, selector, direction)

    # 곱셈 노드이면서 직접 값이 맞는 경우
    if gate.operation == '*' and target.evaluate() == c:
        selector[gate_idx] = 1

def gate_selector(gate_list, direction, transcript_table):
    """게이트에서 left 또는 right selector polynomial을 찾는 함수"""
    polynomial_table = []
    for c in transcript_table:
        selector = [0] * len(gate_list)
        for gate_idx, gate in enumerate(gate_list):
            find_selector(gate, gate_idx, c, selector, direction)
        polynomial_table.append(selector)
    return polynomial_table



In [186]:
def properties(transcript_table, selector_table):
    """주어진 selector 테이블에 대해 property 값을 계산하는 함수"""
    def compute(x):
        return sum(c * s for c, s in zip(transcript_table, selector_table[x]))
    return compute

def master_polynomial(transcript_table, selector_tables):
    """QAP의 master polynomial을 계산하는 클로저"""
    def compute(x):
        left_properties = properties(transcript_table, selector_tables['left'])(x)
        right_properties = properties(transcript_table, selector_tables['right'])(x)
        output_properties = properties(transcript_table, selector_tables['output'])(x)

        return [left_properties,  right_properties, output_properties]

    return compute


In [187]:

def vanishing_polynomial(omega, n):
    def compute(x):
        result = 1
        for i in range(n):
            result *= (x - pow(omega, i + 1))
        return result
    return compute

In [188]:
input_nodes = [
    Node(value=3),
    Node(value=2),
    Node(value=1),
    Node(value=7),
    Node(value=5),
    Node(value=4),
]

gate1 = Node(operation='*', left=input_nodes[0], right=input_nodes[1])
gate2 = Node(operation='*', left=gate1, right=Node(operation='+', left=input_nodes[2], right=input_nodes[3]))
gate3 = Node(operation='*', left=Node(operation='+', left=input_nodes[2], right=input_nodes[3]),  right=Node(operation='+', left=input_nodes[4], right=input_nodes[5]))

transcript_table = [3, 2, 1, 7, 5, 4, 6, 48, 72]
gate_list = [gate1, gate2, gate3]


In [189]:
print("Gate Structure:")
print(f"Gate 1: {gate1}")
print(f"Gate 2: {gate2}")
print(f"Gate 3: {gate3}")

print("\nEvaluations:")
print(f"Gate 1 Result: {gate1.evaluate()}")
print(f"Gate 2 Result: {gate2.evaluate()}")
print(f"Gate 3 Result: {gate3.evaluate()}")

Gate Structure:
Gate 1: ((3) * (2))
Gate 2: (((3) * (2)) * ((1) + (7)))
Gate 3: (((1) + (7)) * ((5) + (4)))

Evaluations:
Gate 1 Result: 6
Gate 2 Result: 48
Gate 3 Result: 72


In [190]:
gate_selector(gate_list, 'left', transcript_table)

[[1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [191]:
gate_selector(gate_list, 'right', transcript_table)

[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [192]:
# 실행 예시 (left selector 생성)
left_selector_table = gate_selector(gate_list, 'left', transcript_table)
print("Left Selector Table:")
left_selector_table

Left Selector Table:


[[1, 0, 0],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 1, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [193]:
# 실행 예시 (left selector 생성)
right_selector_table = gate_selector(gate_list, 'right', transcript_table)
print("Right Selector Table:")
right_selector_table

Right Selector Table:


[[0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]

In [194]:
# 실행 예시 (left selector 생성)
output_selector_table = gate_selector(gate_list, 'output', transcript_table)
print("output Selector Table:")
output_selector_table

output Selector Table:


[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]

# R1CS

In [195]:
selector_tables = {
    'left': left_selector_table,
    'right': right_selector_table,
    'output': output_selector_table
}

L = np.array(selector_tables['left']).T
R = np.array(selector_tables['right']).T
O = np.array(selector_tables['output']).T
witness = np.array(transcript_table).reshape(-1, 1)

In [196]:
print(f"L:\n{L}")
print(f"R:\n{R}")
print(f"O:\n{O}")
print(f"Witness:\n{witness}")

L:
[[1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 1 1 0 0 0 0 0]]
R:
[[0 1 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0]]
O:
[[0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1]]
Witness:
[[ 3]
 [ 2]
 [ 1]
 [ 7]
 [ 5]
 [ 4]
 [ 6]
 [48]
 [72]]


In [197]:
L_matmul_witness = L @ witness
R_matmul_witness = R @ witness
O_matmul_witness = O @ witness

print(f"L @ Witness:\n{L_matmul_witness}")
print(f"R @ Witness:\n{R_matmul_witness}")
print(f"O @ Witness:\n{O_matmul_witness}")
lhs = L_matmul_witness * R_matmul_witness
rhs = O_matmul_witness
print(f"lhs:\n {lhs}")
print(f"rhs:\n {rhs}")
print(f"lhs == rhs:\n {lhs == rhs}")

L @ Witness:
[[3]
 [6]
 [8]]
R @ Witness:
[[2]
 [8]
 [9]]
O @ Witness:
[[ 6]
 [48]
 [72]]
lhs:
 [[ 6]
 [48]
 [72]]
rhs:
 [[ 6]
 [48]
 [72]]
lhs == rhs:
 [[ True]
 [ True]
 [ True]]


## Groth16

In [563]:
def find_nth_root_of_unity(prime, n):
    from sympy import factorint
    #if (prime - 1) % n != 0:
    #    raise ValueError(f"n={n} must be a divisor of p-1={prime-1}")
    g = find_generators(prime)
    if g is None:
        raise ValueError(f"No generator found for prime {prime}")
    return pow(g, (prime-1)//n, prime)

# 또는 확장체 사용 (예: p=23, n=12)
def find_nth_root_of_unity_extended(prime, n, extension=2):
    from sympy import factorint
    # p^(extension) - 1에서 n이 약수인지 확인
    p_power = pow(prime, extension) - 1
    #if p_power % n != 0:
    #    raise ValueError(f"n={n} must be a divisor of p^extension-1={p_power} for extension={extension}")
    g = find_generators(pow(prime, extension) - 1)  # 확장체의 생성원
    return pow(g, p_power // n, pow(prime, extension))

In [564]:
# 입력 및 게이트 설정
input_nodes = [Node(value=3), Node(value=2), Node(value=1), Node(value=7), Node(value=5), Node(value=4)]
gate1 = Node(operation='*', left=input_nodes[0], right=input_nodes[1])
gate2 = Node(operation='*', left=gate1, right=Node(operation='+', left=input_nodes[2], right=input_nodes[3]))
gate3 = Node(operation='*', left=Node(operation='+', left=input_nodes[2], right=input_nodes[3]), right=Node(operation='+', left=input_nodes[4], right=input_nodes[5]))

transcript_table = [3, 2, 1, 7, 5, 4, 6, 48, 72]
gate_list = [gate1, gate2, gate3]

# Selector Tables
left_selector_table = gate_selector(gate_list, 'left', transcript_table)
right_selector_table = gate_selector(gate_list, 'right', transcript_table)
output_selector_table = gate_selector(gate_list, 'output', transcript_table)

selector_tables = {'left': left_selector_table, 'right': right_selector_table, 'output': output_selector_table}

In [565]:
output_selector_table

[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]

In [566]:
witness

array([[ 3],
       [ 2],
       [ 1],
       [ 7],
       [ 5],
       [ 4],
       [ 6],
       [48],
       [72]])

In [567]:
L = np.array(selector_tables['left']).T
R = np.array(selector_tables['right']).T
O = np.array(selector_tables['output']).T
witness = np.array(transcript_table).reshape(-1, 1)

In [568]:
L.T


array([[1, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [569]:
L_matmul_witness = L @ witness
R_matmul_witness = R @ witness
O_matmul_witness = O @ witness

print(f"L @ Witness:\n{L_matmul_witness}")
print(f"R @ Witness:\n{R_matmul_witness}")
print(f"O @ Witness:\n{O_matmul_witness}")
lhs = L_matmul_witness * R_matmul_witness
rhs = O_matmul_witness
print(f"lhs:\n {lhs}")
print(f"rhs:\n {rhs}")
print(f"lhs == rhs:\n {lhs == rhs}")

L @ Witness:
[[3]
 [6]
 [8]]
R @ Witness:
[[2]
 [8]
 [9]]
O @ Witness:
[[ 6]
 [48]
 [72]]
lhs:
 [[ 6]
 [48]
 [72]]
rhs:
 [[ 6]
 [48]
 [72]]
lhs == rhs:
 [[ True]
 [ True]
 [ True]]


In [570]:
def find_prime(start=2, limit=100):
    """ p+1이 3으로 나누어 떨어지는 가장 작은 소수를 찾는다. """
    def is_prime(n):
        if n < 2:
            return False
        if n in (2, 3):
            return True
        if n % 2 == 0 or n % 3 == 0:
            return False
        i = 5
        while i * i <= n:
            if n % i == 0 or n % (i + 2) == 0:
                return False
            i += 6
        return True

    for p in range(start, limit):
        if is_prime(p) and (p + 1) % 3 == 0:
            return p  # 조건을 만족하는 첫 번째 소수를 반환

    return None  # 범위 내에서 조건을 만족하는 소수를 찾지 못한 경우

# 실행 예제
prime_number = find_prime(start=54, limit=1000)
print(f"조건을 만족하는 첫 번째 소수: {prime_number}")


조건을 만족하는 첫 번째 소수: 59


In [572]:
# 타원 곡선 및 매개변수
prime = 13
groth16_curve = EllipticCurve(a=FiniteField(1, prime), b=FiniteField(1, prime), prime=prime) 
n = 3
points_of_order_n = [p for p in groth16_curve.find_points_of_order_n(n) if p is not None]
print(points_of_order_n)
g = points_of_order_n[0]
omega = find_nth_root_of_unity(prime, n)
print(omega)
print(omega ** n % prime)
tau = random.randint(0, 123456789) # 예: 4 (F_7에서 3, 2, 1 외의 값)

[(FiniteField(10, 13), FiniteField(6, 13)), (FiniteField(10, 13), FiniteField(7, 13))]
3
1


In [573]:
x_values = [omega, omega ** 2 % prime, omega ** 3 % prime]
L_poly = []
R_poly = []
O_poly = []
for i in range(len(selector_tables['left'])):
    L_coefficients = np.polyfit(x_values, selector_tables['left'][i], 2)
    R_coefficients = np.polyfit(x_values, selector_tables['right'][i], 2)
    O_coefficients = np.polyfit(x_values, selector_tables['output'][i], 2)
    
    L_poly.append(np.poly1d(L_coefficients))
    R_poly.append(np.poly1d(R_coefficients))
    O_poly.append(np.poly1d(O_coefficients))


In [574]:
def calculate_exponent(witness, poly, tau):
    result = 0
    for i in range(len(witness)):
        result += (witness[i] * poly[i](tau))
        
    return result
    
L_exponent = int(calculate_exponent(witness, L_poly, tau)) % prime
R_exponent = int(calculate_exponent(witness, R_poly, tau)) % prime
O_exponent = int(calculate_exponent(witness, O_poly, tau)) % prime

print(f"L exponent: {L_exponent}")
print(f"R exponent: {R_exponent}")
print(f"O exponent: {O_exponent}")

L exponent: 10
R exponent: 9
O exponent: 11


In [575]:
pi_1 = groth16_curve.multiply_point(g, int(L_exponent) % prime)
pi_2 = groth16_curve.multiply_point(g, int(R_exponent) % prime)
pi_3 = groth16_curve.multiply_point(g, int(O_exponent) % prime)

print(f"pi_1: {pi_1}")
print(f"pi_2: {pi_2}")
print(f"pi_3: {pi_3}")

pi_1: (FiniteField(10, 13), FiniteField(6, 13))
pi_2: None
pi_3: (FiniteField(10, 13), FiniteField(7, 13))


In [576]:


def vanishing_polynomial_prime(omega, n, prime):
    def compute(x):
        result = 1
        for i in range(n):
            result *= (x - pow(omega, i + 1, prime))
        return result % prime
    return compute

print(f"Omega: {omega}, n: {n}, Prime: {prime}")
V_x = vanishing_polynomial_prime(omega, n, prime)
print(tau)
V_tau = V_x(tau)
print(V_tau)
inverse_V_tau = pow(V_tau, prime - 2, prime)
q_tau = ((L_exponent * R_exponent - O_exponent)*inverse_V_tau) % prime

pi_4 = groth16_curve.multiply_point(g, int(q_tau) % prime)
g_V_tau = groth16_curve.multiply_point(g, V_tau)

Omega: 3, n: 3, Prime: 13
21038227
7


In [577]:
print((L_exponent * R_exponent) % prime)
print((O_exponent) % prime)
print((L_exponent * R_exponent - O_exponent) % prime)
print(V_tau * q_tau % prime)

12
11
1
1


In [578]:
print(f"g: {g}")
print(f"g^n: {groth16_curve.multiply_point(g, n)}")  # 항등원이면 None

g: (FiniteField(10, 13), FiniteField(6, 13))
g^n: None


In [579]:
V_tau, q_tau, g_V_tau

(7, 2, (FiniteField(10, 13), FiniteField(6, 13)))

In [580]:
pi_1, pi_2, pi_3, pi_4

((FiniteField(10, 13), FiniteField(6, 13)),
 None,
 (FiniteField(10, 13), FiniteField(7, 13)),
 (FiniteField(10, 13), FiniteField(7, 13)))

In [591]:
extension = 5
extended_groth15_curve = EllipticCurve(a=FiniteFieldExtension(1, 0, prime, extension), b=FiniteFieldExtension(1, 0, prime, extension), prime=prime)
extended_pi_1 = None if pi_1 is None else (pi_1[0].to_extension(extension), pi_1[1].to_extension(extension))
extended_pi_2 = None if pi_2 is None else (pi_2[0].to_extension(extension), pi_2[1].to_extension(extension))
extended_pi_3 = None if pi_3 is None else (pi_3[0].to_extension(extension), pi_3[1].to_extension(extension))
extended_g = (g[0].to_extension(extension), g[1].to_extension(extension))
extended_g_V_tau = None if g_V_tau is None else(g_V_tau[0].to_extension(extension), g_V_tau[1].to_extension(extension))
extended_pi_4 = None if pi_4 is None else (pi_4[0].to_extension(extension), pi_4[1].to_extension(extension))

weil_pairing_1 = WeilPairing(extended_groth15_curve, extended_pi_1, extended_pi_2)
weil_pairing_2 = WeilPairing(extended_groth15_curve, extended_pi_3, extended_g)

result1, _, _ = weil_pairing_1.compute_pairing(n)
result2, _, R_g = weil_pairing_2.compute_pairing(n)

weil_pairing_3 = WeilPairing(extended_groth15_curve, extended_g_V_tau, extended_pi_4, None, None)
result3, _, _ = weil_pairing_3.compute_pairing(n)

print(f"e(pi_1, pi_2): {result1}")
print(f"e(pi_3, g): {result2}")
print(f"e(g_V_tau, pi_4): {result3}")
print(f"e(pi_1, pi_2) / e(pi_3, g) == e(g_V_tau, pi_4): {result1 * result2.inverse()} == {result3} => {result1 / result2 == result3}")

e(pi_1, pi_2): FiniteFieldExtension(1 + 0*alpha, prime=13, alpha^2=5)
e(pi_3, g): FiniteFieldExtension(9 + 0*alpha, prime=13, alpha^2=5)
e(g_V_tau, pi_4): FiniteFieldExtension(9 + 0*alpha, prime=13, alpha^2=5)
e(pi_1, pi_2) / e(pi_3, g) == e(g_V_tau, pi_4): FiniteFieldExtension(3 + 0*alpha, prime=13, alpha^2=5) == FiniteFieldExtension(9 + 0*alpha, prime=13, alpha^2=5) => False


In [589]:
print(f"extended_pi_1: {extended_pi_1}")
print(f"extended_pi_2: {extended_pi_2}")
print(f"extended_pi_3:L {extended_pi_3}")
print(f"extended_g: {extended_g}")
print(f"extended_g_V_tau: {extended_g_V_tau}")
print(f"extended_pi_4: {extended_pi_4}")

extended_pi_1: (FiniteFieldExtension(10 + 0*alpha, prime=13, alpha^2=5), FiniteFieldExtension(6 + 0*alpha, prime=13, alpha^2=5))
extended_pi_2: None
extended_pi_3:L (FiniteFieldExtension(10 + 0*alpha, prime=13, alpha^2=5), FiniteFieldExtension(7 + 0*alpha, prime=13, alpha^2=5))
extended_g: (FiniteFieldExtension(10 + 0*alpha, prime=13, alpha^2=5), FiniteFieldExtension(6 + 0*alpha, prime=13, alpha^2=5))
extended_g_V_tau: (FiniteFieldExtension(10 + 0*alpha, prime=13, alpha^2=5), FiniteFieldExtension(6 + 0*alpha, prime=13, alpha^2=5))
extended_pi_4: (FiniteFieldExtension(10 + 0*alpha, prime=13, alpha^2=5), FiniteFieldExtension(7 + 0*alpha, prime=13, alpha^2=5))
